## Plotly
- https://plotly.com/python/
-  pip install plotly
- import matplotlib.pyplot as plt

In [4]:
import plotly.express as px # epxress에서 예제 데이터 가져오기기
df = px.data.stocks()
df

,date,GOOG,AAPL,AMZN,FB,NFLX,MSFT
0,2018-01-01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,2018-01-08,1.018172,1.011943,1.061881,0.959968,1.053526,1.015988
2,2018-01-15,1.032008,1.019771,1.053240,0.970243,1.049860,1.020524
3,2018-01-22,1.066783,0.980057,1.140676,1.016858,1.307681,1.066561
4,2018-01-29,1.008773,0.917143,1.163374,1.018357,1.273537,1.040708
...,...,...,...,...,...,...,...
100,2019-12-02,1.216280,1.546914,1.425061,1.075997,1.463641,1.720717
101,2019-12-09,1.222821,1.572286,1.432660,1.038855,1.421496,1.752239
102,2019-12-16,1.224418,1.596800,1.453455,1.104094,1.604362,1.784896
103,2019-12-23,1.226504,1.656000,1.521226,1.113728,1.567170,1.802472


In [8]:
fig= px.line(df, x='date', y='GOOG') # fig는 그래프 객체를 의미한다
fig.show()

In [9]:
import plotly.express as px

df=px.data.stocks(indexed=True)
df

company,GOOG,AAPL,AMZN,FB,NFLX,MSFT
date,,,,,,
2018-01-01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2018-01-08,1.018172,1.011943,1.061881,0.959968,1.053526,1.015988
2018-01-15,1.032008,1.019771,1.053240,0.970243,1.049860,1.020524
2018-01-22,1.066783,0.980057,1.140676,1.016858,1.307681,1.066561
2018-01-29,1.008773,0.917143,1.163374,1.018357,1.273537,1.040708
...,...,...,...,...,...,...
2019-12-02,1.216280,1.546914,1.425061,1.075997,1.463641,1.720717
2019-12-09,1.222821,1.572286,1.432660,1.038855,1.421496,1.752239
2019-12-16,1.224418,1.596800,1.453455,1.104094,1.604362,1.784896


In [ ]:
fig = px.area(df,facet_col= 'company', facet_col_wrap=2) 
# area 채워주는 그래프, facet_col = 'company' : 그래프를 나눠서 그려줌, facet_col_wrap : 열을 몇개로 해주냐! 여기선 2개
fig

## 캔들 차트
- https://plotly.com/python/candlestick-charts/
- 1 코드의 데이터를 읽어오는 함수 사용

In [12]:
def stockDataReader_fn(stock_name, start_date = None, end_date =None): 

    import FinanceDataReader as fdr
    import datetime
    import os
    import pandas as pd

    def StockListing():
        
        import requests
        import json

        try:
            from pandas import json_normalize
        except ImportError:
            from pandas.io.json import json_normalize

        #요청시 넘겨줄 pramaters
        params ={
            'bld': 'dbms/MDC/STAT/standard/MDCSTAT01901',
            'locale': 'ko_KR',
            'mktId': 'ALL',
            'share': '1',
            'csvxls_isNo': 'false'
        }
        
        #요청시 헤더 정보
        headers = {
            'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201',
            'Upgrade-Insecure-Requests': '1', 
            'User-Agent': 'Mozilla/5.0'
        }
        
        r = requests.post('http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd', params, headers=headers)
        print(r)
        
        jo = json.loads(r.text)
        df = json_normalize(jo, 'OutBlock_1')
        df_info = df[['ISU_SRT_CD', 'ISU_ABBRV']]
        
        df_info.columns = ['Symbol', 'Name']
        return df_info
    #1. 저장 파일 형식에 필요한 Data 불러오기
    today_year = datetime.datetime.today().year
    today_month = datetime.datetime.today().month
    today_day = datetime.datetime.today().day
    
    # 2. KRX 종목 DataFrame 존재 여부 확인
    if os.path.isfile("krx_df_{}_{}_{}.csv".format(today_year, today_month, today_day)):
        krx_df = pd.read_csv("krx_df_{}_{}_{}.csv".format(today_year, today_month, today_day), index_col = 0)
    else:
        krx_df = StockListing()
        krx_df.to_csv("krx_df_{}_{}_{}.csv".format(today_year, today_month, today_day))
    
    # 3. 종목 코드 찾기
    if stock_name in krx_df['Name'].values:
        symbol = krx_df[krx_df['Name'] == stock_name]['Symbol'].values[0]
    else:
        print("There is no name in KRX")
        raise "Threr is no name in KRX" # 함수가 중간에 멈추고 -> error 발생
    
    # 찾은 심볼을 가지고 -> 시작, 끝 날자의 상장 주식 정보를 반환함 !
    stock_df = fdr.DataReader(symbol, start_date, end_date)
    
    return stock_df





In [13]:
stock_name = '롯데케미칼'
start_time = '2021'

df=stockDataReader_fn(stock_name=stock_name, start_date=start_time)
df

<Response [200]>


,Open,High,Low,Close,Volume,Change
Date,,,,,,
2021-01-04,264796,269083,263844,266226,359518,0.012682
2021-01-05,264320,269559,263368,269083,282037,0.010731
2021-01-06,274321,277655,268606,269559,476734,0.001769
2021-01-07,277179,290514,276226,289562,747074,0.074206
2021-01-08,291943,292419,281941,286705,471704,-0.009867
...,...,...,...,...,...,...
2025-01-20,57600,59300,57600,58300,110831,0.013913
2025-01-21,58800,61100,57800,58700,172528,0.006861
2025-01-22,59300,59400,57800,59100,82721,0.006814


In [ ]:
import plotly.graph_objects as go

candlestick = go.Candlestick(x=df.index,open=df['Open'],high = df['High'], low =df['Low'], close = df['Close'], name = 'CandleStick')
fig = go.Figure(candlestick) # Figure 라는 객체에 넣어줘야 한다다
fig.update_xaxes(rangeslider_visible=False)
fig.show()

## DrawDown그리기

In [32]:
def mdd_fn(df):
    
    df = df[['Close']].copy()
    
    def return_fn(df):
        return df['Close'].pct_change().fillna(0)
    
    def cum_return_fn(df_return):
        return (1 + df_return).cumprod()
    
    df['Return'] = return_fn(df)
    
    df['CumReturn'] = cum_return_fn(df['Return'])
    
    df['MaxCumReturn'] = df['CumReturn'].cummax()
    
    df['DrawDown'] = (df['CumReturn'] / df['MaxCumReturn']) -1
    
    mdd = df['DrawDown'].min()
    
    list_info = list()
    
    # 최저가
    min_close_value = df['Close'].min()
    
    # 최고가
    max_close_value = df['Close'].max()

    print("최고: {}\n최저: {}\nMDD: {}".format(max_close_value, min_close_value,round(mdd*100,2)))
    
    df_max_close = df[df['DrawDown'] == 0].copy()

    df_max_close.loc[df.index[len(df)-1]] = 0

    period = df_max_close.index[1:] - df_max_close.index[:-1]

    mdd_days = period.days

    max_period = mdd_days.max()

    max_period_idx = mdd_days.argmax()

    print("MAX Draw Down: {} ~ {}".format(df_max_close.index[:-1][max_period_idx].date(), df_max_close.index[1:][max_period_idx].date()))
    print("{} days".format(max_period))
    print("max period : {}".format(max_period))
    
    list_info.append(max_close_value)
    list_info.append(min_close_value)
    list_info.append(round(mdd*100,2))
    list_info.append(df_max_close.index[:-1][max_period_idx].date())
    list_info.append(df_max_close.index[1:][max_period_idx].date())
    list_info.append(max_period)
          
    return df, list_info

In [23]:
df_dd, stock_info = mdd_fn(df)
df_dd
stock_info


최고: 312422
최저: 56400
MDD: -81.95
MAX Draw Down: 2021-03-03 ~ 2025-01-24
1423 days
max period : 1423


[np.int64(312422),
 np.int64(56400),
 np.float64(-81.95),
 datetime.date(2021, 3, 3),
 datetime.date(2025, 1, 24),
 np.int64(1423)]

In [22]:
import plotly.graph_objects as go

dd = go.Scatter(x=df_dd.index, y= df_dd['DrawDown'], name= 'Draw Down', fill='tozeroy')
fig = go.Figure(dd)
fig.show()

## MACD 그리기

In [24]:
def macdOscillator_fn(df, short_N, long_N, signal_N):
    
    df = df[['Close']].copy()
    
    df['Short'] = df['Close'].ewm(span = short_N, adjust = False).mean()
    df['Long'] = df['Close'].ewm(span = long_N, adjust = False).mean()
    df['MACD'] = df['Short'] - df['Long']
    df['Signal'] = df['MACD'].ewm(span = signal_N, adjust = False).mean()
    df['MACD Oscillator'] = df['MACD'] - df['Signal']
    
    return df[['MACD', 'Signal', 'MACD Oscillator']]

In [25]:
df_macd = macdOscillator_fn(df, short_N = 9, long_N = 26, signal_N = 13)
df_macd

,MACD,Signal,MACD Oscillator
Date,,,
2021-01-04,0.000000,0.000000,0.000000
2021-01-05,359.770370,51.395767,308.374603
2021-01-06,680.877750,141.321765,539.555986
2021-01-07,3427.544287,610.782125,2816.762162
2021-01-08,5051.563288,1245.179434,3806.383854
...,...,...,...
2025-01-20,-2574.490708,-3102.731645,528.240937
2025-01-21,-2350.877160,-2995.323861,644.446701
2025-01-22,-2100.039315,-2867.426069,767.386753


In [30]:
import plotly.graph_objects as go

macd = go.Scatter(x=df.index, y=df_macd['MACD'], name ="MACD")
signal = go.Scatter(x =df.index,y=df_macd['Signal'], name = 'Signal')
oscillator = go.Bar(x=df.index, y= df_macd['MACD Oscillator'], name='oscillator')

fig = go.Figure(macd)
fig.add_trace(signal)
fig.add_trace(oscillator)

fig.show()

# RSI 그리기

In [33]:
def rsi_fn(df, rsi_period):

    import numpy as np
    
    df_close = df[['Close']].copy()
    
    df_close['Change'] = df_close['Close'] - df_close['Close'].shift(1)
    
    df_close['Gain'] = np.where(df_close['Change']>=0, df_close['Change'], 0)
    
    df_close['Loss'] = np.where(df_close['Change']<0, df_close['Change']*-1, 0)
    
    df_close['avgGain'] = df_close['Gain'].rolling(window = rsi_period).mean()
    
    df_close['avgLoss'] = df_close['Loss'].rolling(window = rsi_period).mean()
    
    df_close['RS'] = df_close["avgGain"] / df_close['avgLoss']
    
    df_close['RSI'] = 100 - (100/(1+df_close['RS']))
    
    return df_close

In [34]:
df_rsi = rsi_fn(df, rsi_period = 14)
df_rsi

,Close,Change,Gain,Loss,avgGain,avgLoss,RS,RSI
Date,,,,,,,,
2021-01-04,266226,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2021-01-05,269083,2857.0,2857.0,0.0,NaN,NaN,NaN,NaN
2021-01-06,269559,476.0,476.0,0.0,NaN,NaN,NaN,NaN
2021-01-07,289562,20003.0,20003.0,0.0,NaN,NaN,NaN,NaN
2021-01-08,286705,-2857.0,0.0,2857.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2025-01-20,58300,800.0,800.0,0.0,300.000000,457.142857,0.656250,39.622642
2025-01-21,58700,400.0,400.0,0.0,328.571429,407.142857,0.807018,44.660194
2025-01-22,59100,400.0,400.0,0.0,357.142857,264.285714,1.351351,57.471264


In [35]:
import plotly.graph_objects as go

rsi = rsi = go.Scatter(x=df_rsi.index, y=df_rsi['RSI'], name="RSI")
fig = go.Figure(rsi)

rsi_low_line = go.Scatter(x=df_rsi.index, y=(df_rsi['RSI']/df_rsi['RSI']).fillna(1)*30,line=dict(color='red',dash='dash'),name='Low bound')
rsi_high_line = go.Scatter(x=df_rsi.index, y=(df_rsi['RSI']/df_rsi['RSI']).fillna(1)*70,line=dict(color='red',dash='dash'),name='High bound')

fig.add_trace(rsi_low_line)
fig.add_trace(rsi_high_line)

fig.show()


In [36]:
df

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2021-01-04,264796,269083,263844,266226,359518,0.012682
2021-01-05,264320,269559,263368,269083,282037,0.010731
2021-01-06,274321,277655,268606,269559,476734,0.001769
2021-01-07,277179,290514,276226,289562,747074,0.074206
2021-01-08,291943,292419,281941,286705,471704,-0.009867
...,...,...,...,...,...,...
2025-01-20,57600,59300,57600,58300,110831,0.013913
2025-01-21,58800,61100,57800,58700,172528,0.006861
2025-01-22,59300,59400,57800,59100,82721,0.006814


In [37]:
volume = go.Bar(x=df.index, y=df["Volume"], name="Volume")
fig = go.Figure(volume)
fig.show()